In [1]:
import pandas as pd
import numpy as np
import time
import tracemalloc
from numba import njit, typed, types

# Define a tuple type for a pair of int64 values.
pair_type = types.UniTuple(types.int64, 2)

# ============================================================
# Numba helper: Compute the bit length of an integer.
# ============================================================
@njit(cache=True)
def get_bit_length(x):
    """
    Returns the bit length of an integer x.
    For example, get_bit_length(5) returns 3 because 5 in binary is '101'.
    """
    length = 0
    while x:
        length += 1
        x = x >> 1
    return length

# ============================================================
# Numba-accelerated exhaustive search functions
# ============================================================
@njit(cache=True)
def exhaustive_search_njit(mask, current_score, comp, n, best_score, best_pairs, current_pairs, allocations):
    """
    Recursively enumerates all pairings (perfect matchings) for n students.
    
    Parameters:
      mask: A bitmask (np.int64) where each set bit indicates that a student is still unmatched.
      current_score: The accumulated compatibility score so far.
      comp: An n x n NumPy array of compatibility scores.
      n: Total number of students.
      best_score: A one-element NumPy array (mutable container) holding the best score found so far.
      best_pairs: A Numba-typed list (of 2-element tuples) that will store the best pairing.
      current_pairs: A Numba-typed list (of 2-element tuples) for the current pairing in the recursion.
      allocations: A one-element NumPy array (of int64) used as a counter for the number of pairing decisions.
    """
    # Base case: all students have been paired.
    if mask == 0:
        if current_score > best_score[0]:
            best_score[0] = current_score
            # Clear best_pairs by popping all elements.
            while len(best_pairs) > 0:
                best_pairs.pop()
            # Copy current_pairs into best_pairs.
            for idx in range(len(current_pairs)):
                best_pairs.append(current_pairs[idx])
        return

    # Choose the first unmatched student using the bit-trick.
    # Instead of mask & -mask .bit_length(), we use get_bit_length().
    i = get_bit_length(mask & -mask) - 1

    # Remove student i from the mask.
    new_mask = mask & ~(1 << i)
    
    # Iterate over candidate partners using a while loop over the bits in new_mask.
    candidate = new_mask
    while candidate:
        # Extract the lowest set bit from candidate to get candidate student j.
        j = get_bit_length(candidate & -candidate) - 1
        # Remove j from candidate so that it is not processed again.
        candidate = candidate & ~(1 << j)
        
        allocations[0] += 1  # Count this pairing decision.
        # Create the next state by removing student j.
        next_mask = new_mask & ~(1 << j)
        current_pairs.append((i, j))
        exhaustive_search_njit(next_mask, current_score + comp[i, j], comp, n, best_score, best_pairs, current_pairs, allocations)
        current_pairs.pop()

@njit(cache=True)
def exhaustive_search_wrapper(comp, n):
    """
    Entry point for the Numba-accelerated exhaustive search.
    
    Parameters:
      comp: An n x n NumPy array of compatibility scores.
      n: Total number of students.
    
    Returns:
      A tuple: (best total compatibility score, best pairing as a typed list, allocation count)
    """
    best_score = np.array([-1e9], dtype=np.float64)  # 1-element array for best score.
    best_pairs = typed.List.empty_list(pair_type)       # Empty typed list for best pairing.
    current_pairs = typed.List.empty_list(pair_type)      # Empty typed list for current pairing.
    allocations = np.zeros(1, dtype=np.int64)            # 1-element array for counting.
    full_mask = np.int64((1 << n) - 1)                    # All students unmatched.
    exhaustive_search_njit(full_mask, 0.0, comp, n, best_score, best_pairs, current_pairs, allocations)
    return best_score[0], best_pairs, allocations[0]

# ============================================================
# Python wrapper function for exhaustive search with Numba
# ============================================================
def solve_roommate_exhaustive(csv_path):
    """
    Loads compatibility data from a CSV file and uses a Numba-accelerated exhaustive search
    to compute the optimal pairing (perfect matching) for the given students.
    
    The function tracks and reports:
      - Optimal pairing (converted to student name pairs)
      - Total compatibility score
      - Number of allocations (pairing decisions)
      - Total runtime
      - Peak memory usage (in MiB)
    
    Parameters:
      csv_path: Path to the CSV file with columns "Student 1", "Student 2", and "Compatibility Score".
    
    Returns:
      A tuple: (best_pairing_named, best_score, allocations, run_time, peak_memory)
    """
    # Load CSV data.
    df = pd.read_csv(csv_path)
    
    # Build a sorted list of unique student names.
    student_names = sorted(list(set(df['Student 1'].tolist() + df['Student 2'].tolist())))
    n = len(student_names)
    
    # Build dictionaries to map student names to indices and vice versa.
    student_to_index = {name: i for i, name in enumerate(student_names)}
    index_to_student = {i: name for i, name in enumerate(student_names)}
    
    # Build the compatibility matrix as an n x n NumPy array.
    comp = np.zeros((n, n), dtype=np.float64)
    for _, row in df.iterrows():
        i = student_to_index[row['Student 1']]
        j = student_to_index[row['Student 2']]
        score = row['Compatibility Score']
        comp[i, j] = score
        comp[j, i] = score  # Ensure symmetry.
    
    # Start timing and memory tracking.
    tracemalloc.start()
    start_time = time.time()
    
    best, best_pairs, allocs = exhaustive_search_wrapper(comp, n)
    
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    run_time = end_time - start_time
    peak_memory = peak / 10**6  # Convert bytes to MiB.
    
    # Convert best_pairs (typed list of index pairs) to student name pairs.
    best_pairing_named = [(index_to_student[p[0]], index_to_student[p[1]]) for p in best_pairs]
    
    print("=== Exhaustive Search with Numba ===")
    print(f"Optimal Pairing: {best_pairing_named}")
    print(f"Total Compatibility Score: {best}")
    print(f"Number of Allocations: {allocs}")
    print(f"Time Taken: {run_time:.4f} seconds")
    print(f"Peak Memory Usage: {peak_memory:.3f} MiB")
    
    return best_pairing_named, best, allocs, run_time, peak_memory

In [3]:
solve_roommate_exhaustive("data/compatibility_6.csv") # Generated after starting kernel new

=== Exhaustive Search with Numba ===
Optimal Pairing: [('Student_1', 'Student_2'), ('Student_3', 'Student_5'), ('Student_4', 'Student_6')]
Total Compatibility Score: 2.2812001621554097
Number of Allocations: 35
Time Taken: 2.7241 seconds
Peak Memory Usage: 25.722 MiB


([('Student_1', 'Student_2'),
  ('Student_3', 'Student_5'),
  ('Student_4', 'Student_6')],
 2.2812001621554097,
 35,
 2.7241430282592773,
 25.72192)

In [2]:
solve_roommate_exhaustive("data/compatibility_6.csv") # Generated after starting kernel new

=== Exhaustive Search with Numba ===
Optimal Pairing: [('Student_1', 'Student_2'), ('Student_3', 'Student_5'), ('Student_4', 'Student_6')]
Total Compatibility Score: 2.2812001621554097
Number of Allocations: 35
Time Taken: 0.4934 seconds
Peak Memory Usage: 19.760 MiB


([('Student_1', 'Student_2'),
  ('Student_3', 'Student_5'),
  ('Student_4', 'Student_6')],
 2.2812001621554097,
 35,
 0.4933938980102539,
 19.759911)

In [2]:
solve_roommate_exhaustive("data/compatibility_10.csv") # Generated after starting kernel new

=== Exhaustive Search with Numba ===
Optimal Pairing: [('Student_1', 'Student_8'), ('Student_10', 'Student_2'), ('Student_3', 'Student_4'), ('Student_5', 'Student_6'), ('Student_7', 'Student_9')]
Total Compatibility Score: 3.708931604508898
Number of Allocations: 2277
Time Taken: 0.5089 seconds
Peak Memory Usage: 19.761 MiB


([('Student_1', 'Student_8'),
  ('Student_10', 'Student_2'),
  ('Student_3', 'Student_4'),
  ('Student_5', 'Student_6'),
  ('Student_7', 'Student_9')],
 3.708931604508898,
 2277,
 0.5089342594146729,
 19.76053)

In [2]:
solve_roommate_exhaustive("data/compatibility_20.csv") # Generated after starting kernel new

=== Exhaustive Search with Numba ===
Optimal Pairing: [('Student_1', 'Student_8'), ('Student_10', 'Student_12'), ('Student_11', 'Student_14'), ('Student_13', 'Student_17'), ('Student_15', 'Student_19'), ('Student_16', 'Student_6'), ('Student_18', 'Student_7'), ('Student_2', 'Student_5'), ('Student_20', 'Student_9'), ('Student_3', 'Student_4')]
Total Compatibility Score: 8.003157676038137
Number of Allocations: 1578346302
Time Taken: 89.1595 seconds
Peak Memory Usage: 19.760 MiB


([('Student_1', 'Student_8'),
  ('Student_10', 'Student_12'),
  ('Student_11', 'Student_14'),
  ('Student_13', 'Student_17'),
  ('Student_15', 'Student_19'),
  ('Student_16', 'Student_6'),
  ('Student_18', 'Student_7'),
  ('Student_2', 'Student_5'),
  ('Student_20', 'Student_9'),
  ('Student_3', 'Student_4')],
 8.003157676038137,
 1578346302,
 89.15950608253479,
 19.760171)

In [ ]:
solve_roommate_exhaustive("data/compatibility_50.csv") # Generated after starting kernel new